### 防止过拟合
有4中方法：  
1. 正则化  
2. Dropout(失活性正则化)  
3. Early stopping: 缺点是不能同时解决J最小和no overfitting  
4. 增大数据集

In [2]:
# 仍然以mnist数据集为例
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings('ignore')

#### 1 先用Dropout进行过拟合处理

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 64
n_batch = mnist.train.num_examples // batch_size
# 先定义3个占位符
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
# 构建一个784-1000-500-10的神经网络
w1 = tf.Variable(tf.truncated_normal([784,1000], stddev=0.1))
b1 = tf.Variable(tf.zeros([1000])+0.1)
z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.tanh(z1)
# 使用dropout正则化
a1_drop = tf.nn.dropout(a1, keep_prob)

w2 = tf.Variable(tf.truncated_normal([1000,500]))
b2 = tf.Variable(tf.zeros([500])+0.01)
a2 = tf.nn.tanh(tf.matmul(a1_drop, w2)+b2)
a2_drop = tf.nn.dropout(a2, keep_prob)

w3 = tf.Variable(tf.truncated_normal([500,10]))
b3 = tf.Variable(tf.zeros([10])+0.01)
a3 = tf.nn.softmax(tf.matmul(a2_drop, w3)+b3)

loss = tf.losses.softmax_cross_entropy(y, a3)
train = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
corrent_pred = tf.equal(tf.argmax(y,1), tf.argmax(a3,1))
accuracy = tf.reduce_mean(tf.cast(corrent_pred, tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.8})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_prob:0.8})
        print('Epoch {}, test accuracy is {}'.format(epoch, acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Epoch 0, test accuracy is 0.7034000158309937
Epoch 1, test accuracy is 0.7491000294685364
Epoch 2, test accuracy is 0.8011999726295471
Epoch 3, test accuracy is 0.8361999988555908
Epoch 4, test accuracy is 0.8402000069618225
Epoch 5, test accuracy is 0.8526999950408936
Epoch 6, test accuracy is 0.859

#### 2 用正则化进行过拟合处理

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 128
n_batch = mnist.train.num_examples // batch_size
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# 构建一个784-1000-500-10的神经网络
w1 = tf.Variable(tf.truncated_normal([784,1000], stddev=0.1))
b1 = tf.Variable(tf.zeros([1000])+0.1)
a1 = tf.nn.tanh(tf.matmul(x,w1)+b1)
a1_drop = tf.nn.dropout(a1, keep_prob)

w2 = tf.Variable(tf.truncated_normal([1000,500], stddev=0.1))
b2 = tf.Variable(tf.zeros([500])+0.1)
a2 = tf.nn.tanh(tf.matmul(a1_drop, w2)+b2)
a2_drop = tf.nn.dropout(a2, keep_prob)

w3 = tf.Variable(tf.truncated_normal([500,10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10])+0.1)
pred = tf.nn.softmax(tf.matmul(a2_drop, w3)+b3)

# 正则化项
l2_loss = tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + \
            tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2) + \
            tf.nn.l2_loss(w3) + tf.nn.l2_loss(b3)
loss = tf.losses.softmax_cross_entropy(y, pred) + 0.0005*l2_loss
# loss = tf.losses.softmax_cross_entropy(y, pred)
train = tf.train.GradientDescentOptimizer(0.3).minimize(loss)

corrent_pred = tf.equal(tf.argmax(y,1), tf.argmax(pred,1))
accuracy = tf.reduce_mean(tf.cast(corrent_pred, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y: mnist.test.labels, keep_prob:1.0})
        print('Epoch {}, test accuracy is {}'.format(epoch, acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Epoch 0, test accuracy is 0.9383999705314636
Epoch 1, test accuracy is 0.9473999738693237
Epoch 2, test accuracy is 0.9531000256538391
Epoch 3, test accuracy is 0.9585999846458435
Epoch 4, test accuracy is 0.9607999920845032
Epoch 5, test accuracy is 0.9599999785423279
Epoch 6, test accuracy is 0.963

对比发现正则化的效果没有Dropout的效果好，说明正则化更适用于深层的网络

本文亮点：  
①过拟合处理方式，有四种：  
1. 正则化；  
2. Dropout(失活性处理)；  
3. early stopping：缺点是不能同时解决J最小和no overfitting;   
4. 增大数据集；  

代码如下：
    
    # 正则化
    ## 在所有层都结束后执行
    l2_loss = tf.nn.l2_loss(w1)+tf.nn.l2_loss(b1)+tf.nn.l2_loss(w2)+tf.nn.l2_loss(b2)+……
    loss = tf.losses.softmax(y, pred) + 0.0005*l2_loss
    
    # Dropout
    keep_prob = tf.placeholder(tf.float32)
    ## 每一层后都执行
    ai_drop = tf.nn.dropout(ai. keep_prob)
    
根据结果发现，正则化更适用于深层次神经网络。